# ***Face-Id detection program***

This program is a Computer Vision Ai program that will predict if the face of the user is their face or its a fraud. 


## Libraries

In [45]:
import numpy as np
import scipy as sp
import os 
import cv2 
import shutil
import random
import torch #pytorch 
import matplotlib.pyplot as plt 
from torchvision import datasets, transforms
import torch.nn as nn #Neural Network
import torch.nn.functional as F #for nn functions(Argmax, Relu, Sigmoid, cross entropy etc...)
import torch.optim as optim # for optimizer Adam
import wandb #for weights and biases to see model train in real time
import random
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
from torchsummary import summary # this torch print summary of model
import sys
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
sys.path.append(os.getcwd())

In [42]:
from digi_face_dataset import DigiFaceDataset

## Data and Organization 

In [8]:
path = os.listdir("/Users/joascerutti/Desktop/InitToWinIt/Faces Data/")
print(os.listdir())

['.DS_Store', 'FaceID.ipynb', 'Split Folder', 'Faces Data']


In [31]:

data_path = "/Users/joascerutti/Desktop/InitToWinIt/Faces Data/"

train_path = os.path.join(data_path, "train")
val_path = os.path.join(data_path, "val")
test_path = os.path.join(data_path, "test")

if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(val_path):
    os.makedirs(val_path)
if not os.path.exists(test_path):
    os.makedirs(test_path)

for folder_name in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder_name)
    if not os.path.isdir(folder_path):
        continue

    train_folder_path = os.path.join(train_path, folder_name)
    val_folder_path = os.path.join(val_path, folder_name)
    test_folder_path = os.path.join(test_path, folder_name)

    if not os.path.exists(train_folder_path):
        os.makedirs(train_folder_path)
    if not os.path.exists(val_folder_path):
        os.makedirs(val_folder_path)
    if not os.path.exists(test_folder_path):
        os.makedirs(test_folder_path)

    images = os.listdir(folder_path)
    random.shuffle(images)

    num_images = len(images)
    train_end = int(num_images * 0.7)
    val_end = int(num_images * 0.85)

    train_images = images[:train_end]
    val_images = images[train_end:val_end]
    test_images = images[val_end:]

    for image_name in train_images:
        src_path = os.path.join(folder_path, image_name)
        dst_path = os.path.join(train_folder_path, image_name)
        if os.path.isfile(src_path):
            shutil.copy(src_path, dst_path)

    for image_name in val_images:
        src_path = os.path.join(folder_path, image_name)
        dst_path = os.path.join(val_folder_path, image_name)
        if os.path.isfile(src_path):
            shutil.copy(src_path, dst_path)

    for image_name in test_images:
        src_path = os.path.join(folder_path, image_name)
        dst_path = os.path.join(test_folder_path, image_name)
        if os.path.isfile(src_path):
            shutil.copy(src_path, dst_path)

class DigiFaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.image_paths = []
        self.labels = []

        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label

In [34]:
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        print(running_loss)
    return running_loss / len(dataloader.dataset)

In [35]:
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
    return running_loss / len(dataloader.dataset)

In [48]:
data_transforms = {
    'train': Compose([
        Resize((224, 224)),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': Compose([
        Resize((224, 224)),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

train_path = '/Users/joascerutti/Desktop/InitToWinIt/Split/train/'
val_path = '/Users/joascerutti/Desktop/InitToWinIt/Split/val/'

train_dataset = DigiFaceDataset(train_path, transform=data_transforms['train'])
val_dataset = DigiFaceDataset(val_path, transform=data_transforms['val'])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=len(train_dataset.classes))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10

Loaded pretrained weights for efficientnet-b0


In [50]:
best_val_loss = float('inf')
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    train_loss = train(model, train_dataloader, criterion, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}")

    val_loss = validate(model, val_dataloader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_efficientnet_b0.pth')
        print("Model saved.")

Epoch 1/10


TypeError: Unexpected type <class 'numpy.ndarray'>